# SMAP-USDA monthly averages

Monthly climatologies/stats of 
soil moisture quantities and 
their derivatives (porosity and available porosity)

James McCreight 


In [1]:
import mon_stats
import ee

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
smap_usda = (ee
  .ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture'))

In [5]:
smap_usda_clim = mon_stats.bands_avgs(['smp', 'ssm', 'susm'], smap_usda)

In [6]:
smap_usda_clim

{'avgs': <ee.dictionary.Dictionary at 0x7fe3789b7cd0>,
 'lens': <ee.dictionary.Dictionary at 0x7fe3789b7c10>}

In [7]:
smap_usda_clim['avgs'].getInfo()

{'smp': {'type': 'ImageCollection', 'bands': []},
 'ssm': {'type': 'ImageCollection', 'bands': []},
 'susm': {'type': 'ImageCollection', 'bands': []}}

In [8]:
smap_usda_clim['lens'].getInfo()

{'smp': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 60],
 'ssm': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 60],
 'susm': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 60]}

In [9]:
# separate the variables
smp = ee.ImageCollection(smap_usda_clim['avgs'].get('smp'))
ssm = ee.ImageCollection(smap_usda_clim['avgs'].get('ssm'))
susm = ee.ImageCollection(smap_usda_clim['avgs'].get('susm'))

In [10]:
months = mon_stats.months_list()

In [11]:
# porosity in mm (millimeters)
def mon_porosity_calc(mm):
  porosity = (ssm
    .filter(ee.Filter.eq('month', mm)).first()
    .add(susm.filter(ee.Filter.eq('month', mm)).first())
    .divide(smp.filter(ee.Filter.eq('month', mm)).first())
    .set('month', mm).set('band', 'porosity_mm'))
  return ee.Image(porosity)

porosity = ee.ImageCollection(months.map(mon_porosity_calc))
# porosity.getInfo()

In [12]:
# available/additional/unused porosity
def mon_avail_porosity_calc(mm):
  avail_porosity = (porosity
    .filter(ee.Filter.eq('month', mm)).first()
    .subtract(
      ssm.filter(ee.Filter.eq('month', mm)).first()
      .add(susm.filter(ee.Filter.eq('month', mm)).first()))
      .set('month', mm).set('band', 'avail_porosity_mm'))
  return ee.Image(avail_porosity)

avail_porosity = ee.ImageCollection(months.map(mon_avail_porosity_calc))
# avail_porosity.getInfo()

In [13]:
# Export to images, build a collection
smap_collection = ee.ImageCollection(
  ee.List([
    ssm.toList(12),
    susm.toList(12),
    smp.toList(12),
    porosity.toList(12),
    avail_porosity.toList(12)]).flatten())

# smap_collection.getInfo()

In [14]:
smap_asset = ee.data.createAsset({'type': 'ImageCollection'}, 'users/jamesmcc/smap_usda_climatology')
#smap_asset = {'id': 'users/jamesmcc/smap_usda_climatology_2'}

In [15]:
smap_asset

{'type': 'IMAGE_COLLECTION',
 'name': 'projects/earthengine-legacy/assets/users/jamesmcc/smap_usda_climatology',
 'id': 'users/jamesmcc/smap_usda_climatology',
 'updateTime': '2021-01-03T21:28:58.630438Z'}

In [16]:
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
bands = ['smp', 'ssm', 'susm', 'porosity_mm', 'avail_porosity_mm']
bbox = ee.Geometry.BBox(-180.0, -90, 180, 90)
for bb in bands:
    for mm0 in range(12):
        mm = mm0 + 1
        description = 'SMAP-USDA_mon_clim_' + str(bb) + '_' + str(mm).zfill(2) + '_' + month_names[mm0]
        xx = ee.batch.Export.image.toAsset(
            smap_collection
            .filter(ee.Filter.eq('month', mm))
            .filter(ee.Filter.eq('band', bb)).first(),
            description=description, 
            assetId=smap_asset['id'] + '/' + description,
            region=bbox).start()